# Analysis / Plotting - Notebook 07 - p_depolym_GDP and p_hyd
Author: Bettina Vignolo

This notebook was created to perform several smaller experiments with the microtubule model and analyze the results. It focuses on ```p_depolym_GDP and p_hyd``` and how these parameter influence MT length and each other


In [1]:
# load Packages:
using Agents
using Statistics
using DataFrames
using Distributions
using StatsBase
using DrWatson: @dict
using Serialization
using InteractiveDynamics
using GLMakie
import ColorSchemes as co
using DataFramesMeta

# load Functions:
include("01_Agent.jl")
include("02_Initialization.jl")
include("03_AgentStep.jl")
include("04_ModelStep.jl")
include("05_Analysis.jl")

MT_size_each (generic function with 1 method)

In [1]:
#for parallelization
using Distributed
addprocs(8)

@everywhere begin
    # load Packages:
    using Agents
    using Statistics
    using DataFrames
    using Distributions
    using StatsBase
    using DrWatson: @dict
    using Serialization
    using InteractiveDynamics
    using GLMakie
    import ColorSchemes as co
    using DataFramesMeta

    # load Functions:
    include("01_Agent.jl")
    include("02_Initialization.jl")
    include("03_AgentStep.jl")
    include("04_ModelStep.jl")
    include("05_Analysis.jl")
end    

### 1.1 p_depolym_GDP = 1.0 (10 Ensemble + 100k steps)
runtime 21 min

In [2]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 1.0)  
    for x in repeat([0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_1_0p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

additional p_hyd = 0.006

In [25]:
@everywhere begin
    models = [initialize(;
            Nstarts = 4,
            periodic = true,
            numagents = 1000,
            griddims = (100,100), 
            p_polym = 1.0,
            p_hyd = x,
            p_GTP_exchange = 0.01,
            p_depolym_GTP = 0.0,
            p_depolym_GDP = 1.0)  
        for x in repeat([0.006], 10)]
    
    mdata = [mean_MT_size, (:p_hyd)]
    mlabels = ["mean MT length", "p_hyd"]
    end
    
    data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
    serialize("data/paramscan_1_0p_depolym_GDP_10ensemble_100kSteps_additional_0_006p_hyd.jls", data_ens[2])

In [3]:
data1_1 = deserialize("data/paramscan_1_0p_depolym_GDP_10ensemble_100kSteps.jls")
gdf = groupby(data1_1, [:step, :p_hyd]) #@subset(data1_1, :ensemble .!= 14)
sum_1_1 = combine(gdf, [:mean_MT_size] .=> [mean]);

In [5]:
#MT length over time
f1_6 = Figure(resolution =(1200, 600))
ax1_6 = Axis(f1_6[1,1]; xlabel = "Timesteps", ylabel = "MT size", title = "Mean MT length over time (10x ensemble) - excluded ensemble 14")
range = [0.012, 0.018, 0.03]

for (i,v) in enumerate(range)
    sub_data = @subset(sum_1_1, :p_hyd .== v)
    lines!(ax1_6, sub_data.step, sub_data.mean_MT_size_mean, label = "$v", color = co.tab20[i])
end


Legend(f1_6[2,1], ax1_6, "p_hyd", orientation = :horizontal, tellwidth = false, tellheight = true)

f1_6

### 1.2 p_depolym_GDP = 0.7 (10 Ensemble + 100k steps)
runtime 28 min

In [6]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.7)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_7p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

In [10]:
@subset(data_07, :p_hyd .==0.006, :step .== 50000)

,step,mean_MT_size,p_hyd,ensemble
,Int64,Float64,Float64,Int64
1,50000,100.0,0.006,1
2,50000,100.0,0.006,5
3,50000,75.0,0.006,9
4,50000,100.0,0.006,13
5,50000,100.0,0.006,17
6,50000,100.0,0.006,21
7,50000,100.0,0.006,25
8,50000,100.0,0.006,29
9,50000,75.0,0.006,33


### 1.3 p_depolym_GDP = 0.2 (10 Ensemble + 100k steps)
runtime 28 min

In [7]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.2)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_2p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 1.4 p_depolym_GDP = 0.5 (10 Ensemble + 100k steps)


In [ ]:
#from "Analysis 02_p_hyd" Notebook:
deserialize("data/paramscan_10ensemble_p_hyd_100kSteps.jls")

### 1.5 p_hyd = 0.024 (10 Ensemble + 100k steps)

In [ ]:
#from "Analysis 01_p_depolym_GDP" Notebook:
deserialize("data/paramscan_p_depolym_GDP_10ensemble_100k_Steps_4Nstarts_0_024_phyd.jls")

### 1.6 p_depolym_GDP = 0.4 (10 Ensemble + 100k steps)
runtime 28 min

In [2]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.4)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
end
data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_4p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 1.7 p_depolym_GDP = 0.6 (10 Ensemble + 100k steps)
runtime 28 min

In [3]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.6)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]
mdata = [mean_MT_size, (:p_hyd)]
end
data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_6p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 1.8 p_depolym_GDP = 0.8 (10 Ensemble + 100k steps)
runtime 28 min

In [4]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.8)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_8p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 1.9 p_depolym_GDP = 0.3 (10 Ensemble + 100k steps)
runtime 28 min

In [2]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.3)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_3p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 1.10 p_depolym_GDP = 0.9 (10 Ensemble + 100k steps)
runtime 28 min

In [3]:
@everywhere begin
models = [initialize(;
        Nstarts = 4,
        periodic = true,
        numagents = 1000,
        griddims = (100,100), 
        p_polym = 1.0,
        p_hyd = x,
        p_GTP_exchange = 0.01,
        p_depolym_GTP = 0.0,
        p_depolym_GDP = 0.9)  
    for x in repeat([0.006, 0.012, 0.018, 0.03], 10)]

mdata = [mean_MT_size, (:p_hyd)]
mlabels = ["mean MT length", "p_hyd"]
end

data_ens = ensemblerun!(models, agent_step!, model_step!, 100000; mdata, parallel = true)
serialize("data/paramscan_0_9p_depolym_GDP_10ensemble_100kSteps.jls", data_ens[2])

### 2.1 Join all data and plot them

In [10]:
#p_depolym_GDP = 0.2; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_02 = deserialize("data/paramscan_0_2p_depolym_GDP_10ensemble_100kSteps.jls")
sum_02 = combine(groupby(data_02, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.3; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_03 = deserialize("data/paramscan_0_3p_depolym_GDP_10ensemble_100kSteps.jls")
sum_03 = combine(groupby(data_03, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.4; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_04 = deserialize("data/paramscan_0_4p_depolym_GDP_10ensemble_100kSteps.jls")
sum_04 = combine(groupby(data_04, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.5; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_05 = deserialize("data/paramscan_10ensemble_p_hyd_100kSteps.jls")
sum_05 = combine(groupby(data_05, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.6; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_06 = deserialize("data/paramscan_0_6p_depolym_GDP_10ensemble_100kSteps.jls")
sum_06 = combine(groupby(data_06, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.7; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_07 = deserialize("data/paramscan_0_7p_depolym_GDP_10ensemble_100kSteps.jls")
sum_07 = combine(groupby(@subset(data_07, :ensemble .!= 9, :ensemble .!= 33), [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.8; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_08 = deserialize("data/paramscan_0_8p_depolym_GDP_10ensemble_100kSteps.jls")
sum_08 = combine(groupby(data_08, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 0.9; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_09 = deserialize("data/paramscan_0_9p_depolym_GDP_10ensemble_100kSteps.jls")
sum_09 = combine(groupby(data_09, [:step, :p_hyd]), [:mean_MT_size] .=> [mean])

#p_depolym_GDP = 1.0; p_hyd = [0.006, 0.012, 0.018, 0.03]
data_10_1 = deserialize("data/paramscan_1_0p_depolym_GDP_10ensemble_100kSteps_additional_0_006p_hyd.jls")
data_10_2 = deserialize("data/paramscan_1_0p_depolym_GDP_10ensemble_100kSteps.jls")
data_10 = append!(data_10_1, data_10_2)
sum_10 = combine(groupby(data_10, [:step, :p_hyd]), [:mean_MT_size] .=> [mean]) 

#p_depolym_GDP = [0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]; p_hyd = 0.024
data_0024 = deserialize("data/paramscan_p_depolym_GDP_10ensemble_100k_Steps_4Nstarts_0_024_phyd.jls")
sum_0024 = combine(groupby(data_0024, [:step, :p_depolym_GDP]), [:mean_MT_size] .=> [mean]);

In [11]:
range_p_hyd = [0.006, 0.012, 0.018, 0.03]
range_p_depolym = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
means_02 = Float64[]
means_03 = Float64[]
means_04 = Float64[]
means_05 = Float64[]
means_06 = Float64[]
means_07 = Float64[]
means_08 = Float64[]
means_09 = Float64[]
means_10 = Float64[]
means_0024 = Float64[]

for (i,v) in enumerate(range_p_hyd)
    df_sub_02 = @subset(sum_02, :step .>= 50000, :p_hyd .== v)  
    append!(means_02, mean(df_sub_02.mean_MT_size_mean))

    df_sub_03 = @subset(sum_03, :step .>= 50000, :p_hyd .== v)  
    append!(means_03, mean(df_sub_03.mean_MT_size_mean))

    df_sub_04 = @subset(sum_04, :step .>= 50000, :p_hyd .== v)  
    append!(means_04, mean(df_sub_04.mean_MT_size_mean))

    df_sub_05 = @subset(sum_05, :step .>= 50000, :p_hyd .== v)  
    append!(means_05, mean(df_sub_05.mean_MT_size_mean))

    df_sub_06 = @subset(sum_06, :step .>= 50000, :p_hyd .== v)  
    append!(means_06, mean(df_sub_06.mean_MT_size_mean))

    df_sub_07 = @subset(sum_07, :step .>= 50000, :p_hyd .== v)  
    append!(means_07, mean(df_sub_07.mean_MT_size_mean))

    df_sub_08 = @subset(sum_08, :step .>= 50000, :p_hyd .== v)  
    append!(means_08, mean(df_sub_08.mean_MT_size_mean))

    df_sub_09 = @subset(sum_09, :step .>= 50000, :p_hyd .== v)  
    append!(means_09, mean(df_sub_09.mean_MT_size_mean))

    df_sub_10 = @subset(sum_10, :step .>= 50000, :p_hyd .== v)  
    append!(means_10, mean(df_sub_10.mean_MT_size_mean))
end

for (i,v) in enumerate(range_p_depolym)
    df_sub_0024 = @subset(sum_0024, :step .>= 50000, :p_depolym_GDP .== v)  
    append!(means_0024, mean(df_sub_0024.mean_MT_size_mean))
end

push!(means_02, means_0024[1])
push!(means_03, means_0024[2])
push!(means_04, means_0024[3])
push!(means_05, means_0024[4])
push!(means_06, means_0024[5])
push!(means_07, means_0024[6])
push!(means_08, means_0024[7])
push!(means_09, means_0024[8])
push!(means_10, means_0024[9])

5-element Vector{Float64}:
 100.0
  68.64666706665867
  14.314566208675826
   4.694152616947661
   7.237212255754884

In [13]:
xs = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ys = [0.006, 0.012, 0.018, 0.03, 0.024]
matrix = [means_02 means_03 means_04 means_05 means_06 means_07 means_08 means_09 means_10]

f2 = Figure()
ax2 = Axis(f2[1,1], ylabel = "p_hyd", xlabel = "p_depolym_GDP", xticks = (xs, ["0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1.0"]), yticks = (ys, ["0.006", "0.012", "0.018", "0.03", "0.024"]))
ylims!(ax2, 0.003, 0.033)
xlims!(ax2, 0.1, 1.1)
colors = append!(co.diverging_gwr_55_95_c38_n256[1:3:150],co.diverging_gwr_55_95_c38_n256[151:2:250])#co.coolwarm[1:100]

for (i,x) in enumerate(xs) 
    for (j,y) in enumerate(ys)
        scatter!(ax2, [x], [y], markersize = 50, color = colors[round(Int64,matrix[j,i])])
    end
end
Colorbar(f2[1, 2], limits = (0, 100), colormap = colors, label = "Mean MT length")
f2